# Projekt Apache Spark

# Wprowadzenie

Wykorzystując ten notatnik jako szablon zrealizuj projekt Apache Spark zgodnie z przydzielonym zestawem. 

Kilka uwag:

* Nie modyfikuj ani nie usuwaj paragrafów *markdown* w tym notatniku, chyba że wynika to jednoznacznie z instrukcji. 
* Istniejące paragrafy zawierające *kod* uzupełnij w razie potrzeby zgodnie z instrukcjami
    - nie usuwaj ich
    - nie usuwaj zawartych w nich instrukcji oraz kodu
    - nie modyfikuj ich, jeśli instrukcje jawnie tego nie nakazują
* Możesz dodawać nowe paragrafy zarówno zawierające kod jak i komentarze dotyczące tego kodu (markdown)

# Treść projektu

Poniżej w paragrafie markdown wstaw tytuł przydzielonego zestawu

# Zestaw 4 – imdb-persons

# Działania wstępne 

Utworzenie kontekstów

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder \
    .appName("PROJECT2") \
    .master("local[12]") \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "24") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 18:26:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc = spark.sparkContext

W poniższym paragrafie uzupełnij polecenia definiujące poszczególne zmienne. 

Pamiętaj abyś:

* w późniejszym kodzie, dla wszystkich cześci projektu, korzystał z tych zdefiniowanych zmiennych. Wykorzystuj je analogicznie jak parametry
* przed ostateczną rejestracją projektu usunął ich wartości, tak aby nie pozostawiać w notatniku niczego co mogłoby identyfikować Ciebie jako jego autora

In [3]:
## DODANE PRZEZ AUTORA
is_cluster = False

In [4]:
# pełna ścieżka do katalogu w zasobniku zawierającego podkatalogi `datasource1` i `datasource4` 
# z danymi źródłowymi
input_dir = "gs://bd-wsb-09-24-rm/projekt2/input" if is_cluster else "./zestaw4/input"

In [5]:
input_dir

'./zestaw4/input'

Nie modyfikuj poniższych paragrafów. Wykonaj je i używaj zdefniowanych poniżej zmiennych jak parametrów Twojego programu.

In [6]:
# NIE ZMIENIAĆ
# ścieżki dla danych źródłowych 
datasource1_dir = input_dir + "/datasource1"
datasource4_dir = input_dir + "/datasource4"

# nazwy i ścieżki dla wyników dla misji głównej 
# część 1 (Spark Core - RDD) 
rdd_result_dir = "/tmp/output1"

# część 2 (Spark SQL - DataFrame)
df_result_table = "output2"

In [7]:
## DODANE PRZEZ AUTORA
if not is_cluster:
    rdd_result_dir = '.' + rdd_result_dir

In [8]:
## DODANE PRZEZ AUTORA
header_result = ['profession', 'primaryName', 'movies']

In [9]:
# NIE ZMIENIAĆ
import os
def remove_file(file):
    if os.path.exists(file):
        os.remove(file)

remove_file("metric_functions.py")
remove_file("tools_functions.py")

In [10]:
# NIE ZMIENIAĆ
import requests
r = requests.get("https://jankiewicz.pl/bigdata/metric_functions.py", allow_redirects=True)
open('metric_functions.py', 'wb').write(r.content)
r = requests.get("https://jankiewicz.pl/bigdata/tools_functions.py", allow_redirects=True)
open('tools_functions.py', 'wb').write(r.content)

3322

In [11]:
# NIE ZMIENIAĆ
%run metric_functions.py
%run tools_functions.py

Poniższe paragrafy mają na celu usunąć ewentualne pozostałości poprzednich uruchomień tego lub innych notatników

In [12]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 1 (Spark Core - RDD) 
delete_dir(spark, rdd_result_dir)

Successfully deleted directory: ./tmp/output1


In [13]:
# NIE ZMIENIAĆ
# usunięcie miejsca docelowego dla część 2 (Spark SQL - DataFrame) 
drop_table(spark, df_result_table)

The table output2 does not exist.
Successfully deleted directory: file:/workspace/spark-warehouse/output2


***Uwaga!***

Uruchom poniższy paragraf i sprawdź czy adres pod którym dostępny Apache Spark Application UI jest poprawny wywołując następny testowy paragraf. 

W razie potrzeby określ samodzielnie poprawny adres pod którym dostępny Apache Spark Application UI

In [14]:
# adres URL, pod którym dostępny Apache Spark Application UI (REST API)
# 
spark_ui_address = extract_host_and_port(spark, "http://localhost:4040")
spark_ui_address

'http://localhost:4040'

In [15]:
# testowy paragraf
test_metrics = get_current_metrics(spark_ui_address)

# Część 1 - Spark Core (RDD)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [16]:
# NIE ZMIENIAĆ
before_rdd_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź rozwiązanie *misji głównej* oparte na *RDD API*. 

Pamiętaj o wydajności Twojego przetwarzania, *RDD API* tego wymaga. 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

### Przygotowanie danych wyjściowych

In [17]:
ds1 = sc.textFile(f"{datasource1_dir}/*")

In [18]:
movies_data_rdd = ds1.map(lambda line: line.split("\t"))

In [19]:
ds4 = sc.textFile(f"{datasource4_dir}/*")

In [20]:
header = ds4.first()
persons_data_rdd = ds4.filter(lambda line: line != header).map(lambda line: line.split("\t"))

### Wyłuskanie pożądanych kolumn

In [21]:
movies_data_rdd = movies_data_rdd.map(lambda x: [x[0], x[2], x[3]])

In [22]:
persons_data_rdd = persons_data_rdd.map(lambda x: [x[0], x[1], x[4]])

### Wyznaczanie 4 najbardziej popularnych profesji

In [23]:
def map_person_professions(person):
    professions = person[-1]
    mapped_professions = []
    
    for profession in professions.split(','):
        if profession not in ('', 'miscellaneous'):
            mapped_professions.append((profession, 1))
    return mapped_professions

In [24]:
professions_sorted_by_popularity = persons_data_rdd.flatMap(map_person_professions).reduceByKey(lambda x, y: x + y)\
                                                                                    .sortBy(lambda entry: entry[1], False)

In [25]:
top4_popular_professions = tuple(key for key, _ in professions_sorted_by_popularity.take(4))
top4_popular_professions

('actor', 'actress', 'producer', 'writer')

### Filtrowanie wpisów filmów z pełną obsadą

In [26]:
movies_grouped = movies_data_rdd.map(lambda x: [x[0], x[1:]]).groupByKey().mapValues(list)

Funkcja do filtrowania filmów, dla każdego sprawdza czy ma 1.aktora/aktorkę 2.reżysera 3.dwie inne role

In [27]:
import logging as log

def is_fully_casted_rdd(x):
    movie_entries = list(x[1])
    has_actor, has_director, has_other1, has_other2 = False, False, False, False
    other_role = ''
    
    for entry in movie_entries:
        if has_actor and has_director and has_other1 and has_other2:
            return True;
           
        role = entry[1]

        if role == '':
            log.error("Invalid role!")
        
        if role in ('actor', 'actress','self'):
            has_actor = True
            continue
        elif role == 'director':
            has_director = True
            continue
        elif not has_other1:
            has_other1 = True
            other_role = role
            continue
        elif not has_other2 and role not in ('actor', 'actress', other_role):
            has_other2 = True
            continue
    return has_actor and has_director and has_other1 and has_other2

In [28]:
full_cast_movies_grouped = movies_grouped.filter(is_fully_casted_rdd)

In [29]:
full_cast_movies = full_cast_movies_grouped.flatMap(lambda x: [(value[0], value[1], x[0]) for value in x[1]])

### Filtrowanie wpisów filmów dla 4 najpopularniejszych profesji

In [30]:
full_cast_movies_of_top4_roles =  full_cast_movies.filter(lambda x: x[-2] in top4_popular_professions)

### Wyznaczanie 3 najbardziej zaangażowanych osób dla 4 najpopularniejszych profesji

In [31]:
persons_involvement = full_cast_movies_of_top4_roles.distinct().map(lambda x: [tuple(x[:-1]), 1]).reduceByKey(lambda x, y: x + y)\
                                                                                                .map(lambda x: [*x[0],x[1]]) #rozpakowanie klucza

In [32]:
top_3_persons_of_top4_roles_grouped = persons_involvement.groupBy(lambda x: x[1]) \
                                        .mapValues(lambda role_rows: sorted(role_rows, key= lambda x: x[2], reverse=True)[:3])

In [33]:
top_3_persons_of_top4_roles = top_3_persons_of_top4_roles_grouped.flatMap(lambda x: [(person[0], (x[0], person[2])) for person in x[1]])

### Odczytanie nazwisk najbardziej zaangażowanych i formatowanie wyników

In [34]:
person_id_name = persons_data_rdd.map(lambda x: x[:-1])

In [35]:
result = sc.parallelize([header_result]) + top_3_persons_of_top4_roles.leftOuterJoin(person_id_name)\
                                                    .map(lambda x: [x[1][0][0], x[1][1], x[1][0][1]])\
                                                    .sortBy(lambda x: (x[0],-x[-1]))

In [36]:
result.collect()

[['profession', 'primaryName', 'movies'],
 ['actor', 'Luis Eduardo Motoa', 3559],
 ['actor', 'Ronit Roy', 2602],
 ['actor', 'Dilip Joshi', 2385],
 ['actress', 'Luz Stella Luengas', 3636],
 ['actress', 'Rohini Hattangadi', 3240],
 ['actress', 'Kavita Lad', 3204],
 ['producer', 'Shobha Kapoor', 11833],
 ['producer', 'Ekta Kapoor', 8826],
 ['producer', 'Valentin Pimstein', 6081],
 ['writer', 'Tony Warren', 6153],
 ['writer', 'Delia Fiallo', 6132],
 ['writer', 'Sampurn Anand', 5205]]

In [37]:
result.toDF(['role','name','movie_count']).show()

+----------+------------------+-----------+
|      role|              name|movie_count|
+----------+------------------+-----------+
|profession|       primaryName|     movies|
|     actor|Luis Eduardo Motoa|       3559|
|     actor|         Ronit Roy|       2602|
|     actor|       Dilip Joshi|       2385|
|   actress|Luz Stella Luengas|       3636|
|   actress| Rohini Hattangadi|       3240|
|   actress|        Kavita Lad|       3204|
|  producer|     Shobha Kapoor|      11833|
|  producer|       Ekta Kapoor|       8826|
|  producer| Valentin Pimstein|       6081|
|    writer|       Tony Warren|       6153|
|    writer|      Delia Fiallo|       6132|
|    writer|     Sampurn Anand|       5205|
+----------+------------------+-----------+



### Zapisanie wyniku

In [38]:
result.saveAsPickleFile(rdd_result_dir)

### Metryki

Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [39]:
# NIE ZMIENIAĆ
after_rdd_metrics = get_current_metrics(spark_ui_address)

# Część 2 - Spark SQL (DataFrame)

## Misje poboczne

W ponizszych paragrafach wprowadź swoje rozwiązania *misji pobocznych*, o ile **nie** chcesz, aby oceniana była *misja główna*. W przeciwnym przypadku **KONIECZNIE** pozostaw je **puste**.  

## Misja główna 

Poniższy paragraf zapisuje metryki przed uruchomieniem Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [40]:
# NIE ZMIENIAĆ
before_df_metrics = get_current_metrics(spark_ui_address)

W poniższych paragrafach wprowadź rozwiązanie *misji głównej* swojego projektu oparte o *DataFrame API*. 

Pamiętaj o wydajności Twojego przetwarzania, *DataFrame API* nie jest w stanie wszystkiego "naprawić". 

Nie wprowadzaj w poniższych paragrafach żadnego kodu, w przypadku wykorzystania *misji pobocznych*.

### Załadowanie danych i wyłuskanie pożądanych kolumn

In [41]:
from pyspark.sql.functions import *
from pyspark.sql import Window

#### Flmy

In [42]:
ds1_df = spark.read \
    .option("header", "false") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .csv(datasource1_dir)

In [43]:
movies_df = ds1_df.select(col(ds1_df.columns[0]).alias('movieID'), col(ds1_df.columns[2]).alias('personID'), \
                          col(ds1_df.columns[3]).alias('role'))

#### Osoby

In [44]:
ds4_df = spark.read \
    .option("header", "True") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .csv(datasource4_dir)

In [45]:
persons_df = ds4_df.select(expr("nconst as personID"), expr("primaryName as name"), expr('primaryProfession as professions'))

### Wyznaczanie 4 najbardziej popularnych profesji

In [46]:
top4_popular_professions = persons_df.withColumn('profession', explode(split(col('professions'), ","))) \
                        .drop('professions') \
                        .filter(col('profession') != 'miscellaneous') \
                        .groupBy('profession').agg(count('*').alias('count')) \
                        .sort(col('count').desc()).select('profession').limit(4)
top4_professions_list = [row['profession'] for row in top4_popular_professions.collect()]

### Filtrowanie wpisów filmów z pełną obsadą

In [47]:
basic_roles = ( 'director', 'actor', 'actress', 'self' )

def is_df_fully_casted(roles):
        return array_contains(roles, basic_roles[0]) & \
            (array_contains(roles, basic_roles[1]) | array_contains(roles, basic_roles[2]) | array_contains(roles, basic_roles[3])) & \
            (size(array_except(roles,  array( [lit(x) for x in basic_roles] ) )) >= 2)

In [48]:
full_cast_movieIDs = movies_df.groupBy('movieID').agg(collect_set('role').alias('roles'))\
                                .filter(is_df_fully_casted(col('roles'))).drop('roles')

In [49]:
full_cast_movies = movies_df.join(full_cast_movieIDs, on="movieID", how="inner")

### Filtrowanie wpisów filmów dla 4 najpopularniejszych profesji

In [50]:
top4_prof_full_cast_movies = full_cast_movies.filter(col('role').alias('profession').isin( top4_professions_list ))

### Wyznaczanie 3 najbardziej zaangażowanych osób dla 4 najpopularniejszych profesji

In [51]:
movies_per_role_n_person = top4_prof_full_cast_movies.groupBy('role', 'personID').agg(count('*').alias('movies'))

In [52]:
role_window_spec = Window.partitionBy("role").orderBy(col("movies").desc())

ranked_df = movies_per_role_n_person.withColumn("rank", row_number().over(role_window_spec))

top3_persons_of_top4_roles = ranked_df.filter(col("rank") <= 3).drop("rank")

### Odczytanie nazwisk najbardziej zaangażowanych i formatowanie wyników

In [53]:
names_per_IDs = persons_df.drop('professions')

In [54]:
result = top3_persons_of_top4_roles.join(names_per_IDs, on='personID', how='inner').drop('personID').sort(col('role'), col('movies').desc())

In [55]:
result = result.select(col('role').alias(header_result[0]), col('name').alias(header_result[1]), col('movies'))
result.show(truncate=False)

25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:30:22 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+------------------+------+
|profession|primaryName       |movies|
+----------+------------------+------+
|actor     |Luis Eduardo Motoa|3559  |
|actor     |Ronit Roy         |2602  |
|actor     |Dilip Joshi       |2385  |
|actress   |Luz Stella Luengas|3636  |
|actress   |Rohini Hattangadi |3240  |
|actress   |Kavita Lad        |3204  |
|producer  |Shobha Kapoor     |11833 |
|producer  |Ekta Kapoor       |8826  |
|producer  |Valentin Pimstein |6081  |
|writer    |Tony Warren       |6153  |
|writer    |Delia Fiallo      |6132  |
|writer    |Sampurn Anand     |5205  |
+----------+------------------+------+



In [56]:
result.write.saveAsTable(df_result_table)

25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/07 18:31:08 WARN RowBasedKeyValueBatch: Calling spill() on

### Metryki

Poniższy paragraf zapisuje metryki po uruchomieniu Twojego rozwiązania *misji głównej*. 

Nie musisz go uruchamiać podczas implementacji rozwiązania.

In [57]:
# NIE ZMIENIAĆ
after_df_metrics = get_current_metrics(spark_ui_address)

# Analiza wyników i wydajności *misji głównych*

## Część 1 - Spark Core (RDD)

In [58]:
# Wczytanie wyników z pliku pickle
word_counts = sc.pickleFile(rdd_result_dir)

# Wyświetlenie 50 pierwszych elementów
result_sample = word_counts.take(50)
for item in result_sample:
    print(item)

['profession', 'primaryName', 'movies']
['actor', 'Luis Eduardo Motoa', 3559]
['actor', 'Ronit Roy', 2602]
['actor', 'Dilip Joshi', 2385]
['actress', 'Luz Stella Luengas', 3636]
['actress', 'Rohini Hattangadi', 3240]
['actress', 'Kavita Lad', 3204]
['producer', 'Shobha Kapoor', 11833]
['producer', 'Ekta Kapoor', 8826]
['producer', 'Valentin Pimstein', 6081]
['writer', 'Tony Warren', 6153]
['writer', 'Delia Fiallo', 6132]
['writer', 'Sampurn Anand', 5205]


In [59]:
subtract_metrics(after_rdd_metrics, before_rdd_metrics)

{
  "numTasks": 8195,
  "numActiveTasks": 0,
  "numCompleteTasks": 1363,
  "numFailedTasks": 0,
  "numKilledTasks": 0,
  "numCompletedIndices": 1363,
  "executorDeserializeTime": 2335,
  "executorDeserializeCpuTime": 2426209500,
  "executorRunTime": 1622937,
  "executorCpuTime": 60184457700,
  "resultSize": 2813363,
  "jvmGcTime": 26796,
  "resultSerializationTime": 152,
  "memoryBytesSpilled": 0,
  "diskBytesSpilled": 0,
  "peakExecutionMemory": 0,
  "inputBytes": 2794860211,
  "inputRecords": 55863660,
  "outputBytes": 14318,
  "outputRecords": 13,
  "shuffleRemoteBlocksFetched": 0,
  "shuffleLocalBlocksFetched": 37660,
  "shuffleFetchWaitTime": 2,
  "shuffleRemoteBytesRead": 0,
  "shuffleRemoteBytesReadToDisk": 0,
  "shuffleLocalBytesRead": 1474880351,
  "shuffleReadBytes": 1474880351,
  "shuffleReadRecords": 504317,
  "shuffleWriteBytes": 1096994969,
  "shuffleWriteTime": 8619191915,
  "shuffleWriteRecords": 375028
}


## Część 2 - Spark SQL (DataFrame)

In [60]:
df = spark.table(df_result_table)

# Wyświetlenie 50 pierwszych rekordów
df.show(50)

+----------+------------------+------+
|profession|       primaryName|movies|
+----------+------------------+------+
|     actor|Luis Eduardo Motoa|  3559|
|     actor|         Ronit Roy|  2602|
|     actor|       Dilip Joshi|  2385|
|   actress|Luz Stella Luengas|  3636|
|   actress| Rohini Hattangadi|  3240|
|   actress|        Kavita Lad|  3204|
|  producer|     Shobha Kapoor| 11833|
|  producer|       Ekta Kapoor|  8826|
|  producer| Valentin Pimstein|  6081|
|    writer|       Tony Warren|  6153|
|    writer|      Delia Fiallo|  6132|
|    writer|     Sampurn Anand|  5205|
+----------+------------------+------+



In [61]:
subtract_metrics(after_df_metrics, before_df_metrics)

{
  "numTasks": 925,
  "numActiveTasks": 0,
  "numCompleteTasks": 363,
  "numFailedTasks": 0,
  "numKilledTasks": 0,
  "numCompletedIndices": 363,
  "executorDeserializeTime": 1603,
  "executorDeserializeCpuTime": 1237134100,
  "executorRunTime": 1163262,
  "executorCpuTime": 986079871800,
  "resultSize": 1279069,
  "jvmGcTime": 12405,
  "resultSerializationTime": 0,
  "memoryBytesSpilled": 8861508864,
  "diskBytesSpilled": 1057919814,
  "peakExecutionMemory": 9180640672,
  "inputBytes": 10338831663,
  "inputRecords": 190174572,
  "outputBytes": 1360,
  "outputRecords": 12,
  "shuffleRemoteBlocksFetched": 0,
  "shuffleLocalBlocksFetched": 2608,
  "shuffleFetchWaitTime": 0,
  "shuffleRemoteBytesRead": 0,
  "shuffleRemoteBytesReadToDisk": 0,
  "shuffleLocalBytesRead": 2821601250,
  "shuffleReadBytes": 2821601250,
  "shuffleReadRecords": 142885723,
  "shuffleWriteBytes": 2589519417,
  "shuffleWriteTime": 8937574101,
  "shuffleWriteRecords": 133225336
}
